In [1]:
from dotenv  import find_dotenv, load_dotenv
import os
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY)

In [18]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Order(BaseModel):
    """Information about a trading order."""

    # ^ Doc-string for the entity Order.
    # This doc-string is sent to the LLM as the description of the schema Order,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    action: Optional[str] = Field(default=None, description="buy, sell or hold")
    ticker: Optional[str] = Field(default=None, description="ticker you want to trade")
    quantity: Optional[int] = Field(default=None, description="number of units to be traded")
    weight: Optional[float] = Field(default=None, description="weight of the total portfolio, usually in percentages")
    
class Orders(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    orders: List[Order]

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [6]:
import uuid
from typing import Dict, List, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
from langchain_core.pydantic_v1 import BaseModel, Field


class Example(TypedDict):
    """A representation of an example consisting of text input and expected tool calls.

    For extraction, the tool calls are represented as instances of pydantic model.
    """

    input: str  # This is the example text
    tool_calls: List[BaseModel]  # Instances of pydantic model that should be extracted


def tool_example_to_messages(example: Example) -> List[BaseMessage]:
    """Convert an example into a list of messages that can be fed into an LLM.

    This code is an adapter that converts our example to a list of messages
    that can be fed into a chat model.

    The list of messages per example corresponds to:

    1) HumanMessage: contains the content from which content should be extracted.
    2) AIMessage: contains the extracted information from the model
    3) ToolMessage: contains confirmation to the model that the model requested a tool correctly.

    The ToolMessage is required because some of the chat models are hyper-optimized for agents
    rather than for an extraction use case.
    """
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "type": "function",
                "function": {
                    # The name of the function right now corresponds
                    # to the name of the pydantic model
                    # This is implicit in the API right now,
                    # and will be improved over time.
                    "name": tool_call.__class__.__name__,
                    "arguments": tool_call.json(),
                },
            }
        )
    messages.append(
        AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
    )
    tool_outputs = example.get("tool_outputs") or [
        "You have correctly called this tool."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages

In [19]:
examples = [
    (
        "Buy 250 AAPL",
        Order(action="buy", quantity=250, ticker="aapl"),
    ),
    (
        "Sell 500 TSLA",
        Order(action="sell", ticker="tsla", quantity=500),
    ),
    (
        "Roll 500 MSFT",
        Order(action="roll", ticker="msft", quantity=500),
    ),
    (
        "Increase exposure to aapl by 0.5%",
        Order(action="buy",ticker="aapl",quantity=None,weight=0.5)
    ),
    (
        "Decrease exposure to aapl by 0.5%",
        Order(action="sell",ticker="aapl",quantity=None,weight=0.5)
    )
]


messages = []

for text, tool_call in examples:
    messages.extend(
        tool_example_to_messages({"input": text, "tool_calls": [tool_call]})
    )

In [20]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
runnable = prompt | model.with_structured_output(
    schema = Orders,
    method = 'function_calling',
    include_raw = False)

In [21]:
text = "I like to buy wmt stock at $500 price"
print(runnable.invoke({"text": text, "examples": []}))

orders=[Order(action='buy', ticker='WMT', quantity=1, weight=None)]


In [22]:
text = "I want to increase my exposure to meta by 5000 stocks"
print(runnable.invoke({"text": text, "examples": []}))

orders=[Order(action='buy', ticker='META', quantity=5000, weight=None)]


In [23]:
text = "I want to decrease my exposure to rivn by 5000 stocks"
print(runnable.invoke({"text": text, "examples": []}))

orders=[Order(action='sell', ticker='rivn', quantity=5000, weight=None)]


In [26]:
text = "I want to decrease my exposure to rivn, tsla and lcid by 0.5% "
print(runnable.invoke({"text": text, "examples": []}))

orders=[Order(action='sell', ticker='rivn', quantity=None, weight=0.5), Order(action='sell', ticker='tsla', quantity=None, weight=0.5), Order(action='sell', ticker='lcid', quantity=None, weight=0.5)]
